In [10]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [2]:
import re

def _clean_agency_fee(s) -> float:
    if type(s) == float:
        return s
    elif type(s) == str:
        if s == "":
            return 0
        else:
            return float(re.sub('[^0-9\.]','', s))
    else:
        return None

class Publisher(object):
    def __init__(self, id, phone, name, athome_id):
        self.id = id
        self.phone = phone
        self.name = name
        self.athome_id = athome_id

    def __init__(self, id, remote_visit, phone, name, athome_id):
        self.id = id
        self.remoteVisit = remote_visit
        self.phone = phone
        self.name = name
        self.athome_id = athome_id


class Appartment(object):
    def __init__(self, data):
        self.found = data[0]
        self.listingId = data[1]
        self.isSoldProperty = data[3]
        self.region = data[4]
        self.cityName = data[5]
        self.lon = data[6]
        self.lat = data[7]
        self.price = data[8]
        self.chargesPrice = data[9]
        self.caution = data[10]
        self.agencyFee = _clean_agency_fee(data[11])
        self.propertySubType = data[12]
        self.publisher = Publisher(data[13], data[14], data[15], data[16], data[17])
        self.propertySurface = data[18]
        self.buildingYeay = data[19]
        self.floorNumber = data[20]
        self.bathroomsCount = data[21]
        self.bedroomsCount = data[22]
        self.balconiesCount = data[23]
        self.garagesCount = data[24]
        self.carparkCunt = data[25]
        self.hasLivingRoom = data[26]
        self.hasKitchen = data[27]
        self.availability = data[28]
        self.media = data[29]
        self.description = data[30]
        self.link = data[31]
        self.createdAt = data[32]
        self.updatedAt = data[33]

    def _popup_(self):
        return f"""
        <h4>Info</h4>
        <b>ID: </b>{self.listingId}<br>
        <b>Monthly Price: </b>{self.price}<br>
        <b>Monthly Charge: </b>{self.chargesPrice}<br>
        <b>Caution: </b>{self.caution}<br>
        <b>Agency Fee: </b>{self.agencyFee}
        <br>
        <h4>Total</h4>
        <b>Monthly: </b>{self.price + self.chargesPrice}<br>
        <b>Initial: </b>{self.caution + self.agencyFee}
        """

In [13]:
def getAppartments(db):
    cur = db.cursor()
    cur.execute(
        """
        SELECT * 
        FROM apartment 
        WHERE found = TRUE
        """
    )

    rows = cur.fetchall()

    apartmens = []
    for row in rows:
        apartmens.append(Appartment(row))

    return apartmens

In [14]:
import os
import sqlite3

db = sqlite3.connect(os.environ["DB_PATH"])
apartmens = getAppartments(db)
[a.listingId for a in apartmens]

[6467588, 7400265, 7483756, 7491189]

In [15]:
import folium
from folium.plugins import MarkerCluster

lux_coords = [49.611622, 6.131935]
map = folium.Map(location = lux_coords, zoom_start = 10)

amz_coords = [49.630033, 6.168936]
folium.Marker(location=amz_coords, popup="Amazon", icon=folium.Icon(color='red', icon='info-sign')).add_to(map)

folium.Circle(location=amz_coords, radius=5000, color='green', opacity=0.5, weight=2).add_to(map)
folium.Circle(location=amz_coords, radius=10000, color='yellow', opacity=0.5, weight=2).add_to(map)
folium.Circle(location=amz_coords, radius=15000, color='orange', opacity=0.5, weight=2).add_to(map)
folium.Circle(location=amz_coords, radius=20000, color='red', opacity=0.5, weight=2).add_to(map)

for a in apartmens:
    popup = folium.Popup(a._popup_(), max_width=450)
    folium.Marker(location=[a.lat, a.lon], popup=popup).add_to(map)

map